# AIPI 590 - XAI | Explainable Techniques II
### Exploratory analysis of a dataset to determine the amount of correlation between features
### Mariam Khan

[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Mkhan13/XAI/blob/explainable_ml/explainable_techniques_II.ipynb)

In [8]:
import os
import pandas as pd

repo_name = "XAI"
branch_name = "explainable_ml"
git_path = "https://github.com/Mkhan13/XAI.git"

# If repo already exists, remove it first to avoid conflicts
if os.path.exists(repo_name):
    !rm -rf {repo_name}

# Clone the branch
!git clone -b {branch_name} {git_path}

# Path to data folder
data_dir = os.path.join(repo_name, "data", "explainable_ml_II")

# Verify files exist
print("Files in data folder:", os.listdir(data_dir))

Cloning into 'XAI'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 90 (delta 18), reused 11 (delta 11), pack-reused 61 (from 1)
Receiving objects: 100% (90/90), 1.22 MiB | 8.37 MiB/s, done.
Resolving deltas: 100% (33/33), done.
Files in data folder: ['zoo.csv', 'class.csv']


In [11]:
# Load CSV
zoo_path = os.path.join(data_dir, "zoo.csv")
class_path = os.path.join(data_dir, "class.csv")

zoo_df = pd.read_csv(zoo_path)
class_df = pd.read_csv(class_path)

# Merge on class_type
zoo_df = zoo_df.merge(class_df, how="left", left_on="class_type", right_on="Class_Number")

# Drop the redundant numeric columns
zoo_df = zoo_df.drop(columns=["class_type", "Class_Number", "Number_Of_Animal_Species_In_Class", "Animal_Names"])

# Rename for clarity
zoo_df = zoo_df.rename(columns={"Class_Type": "class_type"})

# Show dataset
print(zoo_df.head())

  animal_name  hair  feathers  eggs  milk  airborne  aquatic  predator  \
0    aardvark     1         0     0     1         0        0         1   
1    antelope     1         0     0     1         0        0         0   
2        bass     0         0     1     0         0        1         1   
3        bear     1         0     0     1         0        0         1   
4        boar     1         0     0     1         0        0         1   

   toothed  backbone  breathes  venomous  fins  legs  tail  domestic  catsize  \
0        1         1         1         0     0     4     0         0        1   
1        1         1         1         0     0     4     1         0        1   
2        1         1         0         0     1     0     1         0        0   
3        1         1         1         0     0     4     0         0        1   
4        1         1         1         0     0     4     1         0        1   

  class_type  
0     Mammal  
1     Mammal  
2       Fish  
3     Ma

### **PDP Plot**

*analysis of plots*

### **ICE Plot**

*analysis of plots*

### **ALE Plot**

*analysis of plots*

### **Analysis**

*discussion of findings*